In [1]:
from utils import *

AC (%): 0.53
PR (%): 0.70
RC (%): 0.68
HL    : 5.56
AC (%): 0.61
PR (%): 0.77
RC (%): 0.74
HL    : 4.57
AC (%): 0.61
PR (%): 0.78
RC (%): 0.74
HL    : 4.54
AC (%): 0.60
PR (%): 0.76
RC (%): 0.74
HL    : 4.70


In [2]:
def k_closest(db, query, k=6, norm='l2'):
    if norm == 'l0':
        def key(p): return np.linalg.norm(p[1]-query, ord=0)
    if norm == 'l1':
        def key(p): return np.linalg.norm(p[1]-query, ord=1)
    elif norm == 'l2':
        def key(p): return np.linalg.norm(p[1]-query)
    elif norm == 'linf':
        def key(p): return np.linalg.norm(p[1]-query, ord=np.inf)

    db = [i[0] for i in sorted(enumerate(list(db)), key=key)]
    return db[:k]

In [3]:
def score(db_clss, result):
    # number of retrieved images (-1, because the first one is the query)
    R = len(result) - 1

    # category labels associated with query image
    Lq = db_clss[result[0]]

    # category labels associated with retrieved images
    LXR = [db_clss[i] for i in result[1:]]

    # set of category labels associated to archive
    LX = 4

    ac, pr, rc, hl = 0, 0, 0, 0
    for LXr in LXR:
        intersect = len(
            [1 for index, label in enumerate(LXr) if label == 1 and Lq[index] == 1])
        union = len(
            [1 for index, label in enumerate(LXr) if label == 1 or Lq[index] == 1])
        ac += intersect / union
        pr += intersect / len([1 for l in LXr if l == 1])
        rc += intersect / len([1 for l in Lq if l == 1])
        hl += len([1 for index, label in enumerate(LXr) if (label ==
                                                            1 and Lq[index] == 0) or (label == 0 and Lq[index] == 1)])
    return ac/R, pr/R, rc/R, hl/R

In [4]:
def inference(query, db_feature=None, db_names=None, db_clss=None, model='hsi_model', norm='l2', verbose=False):
    if not db_names:
        db_img, _, db_names, db_clss = get_hsi()
        model = models.load_model(model)
        features = models.Model(
            inputs=model.input, outputs=model.layers[-2].output)
        db_feature = features.predict(db_img)

    query_feature = db_feature[db_names.index(query)]
    closest = k_closest(db_feature, query_feature, norm=norm)
    ac, pr, rc, hl = score(db_clss, closest)
    if verbose:
        print('Retrieved images: ')
        for i in closest:
            print('\t>' + db_names[i][:-3] + 'bmp')
        print('AC (%): {:.2f}\nPR (%): {:.2f}\nRC (%): {:.2f}\nHL    : {:.2f}'.format(
            ac, pr, rc, hl))
    return ac, pr, rc, hl

In [5]:
def cbir(model='hsi_model', norm='l2'):
    db_img, _, db_names, db_clss = get_hsi()
    model = models.load_model(model)
    features = models.Model(
        inputs=model.input, outputs=model.layers[-2].output)
    db_feature = features.predict(db_img)

    AC, PR, RC, HL = 0, 0, 0, 0
    for query in db_names:
        ac, pr, rc, hl = inference(query, db_feature, db_names, db_clss, model, norm)
        AC += ac
        PR += pr
        RC += rc
        HL += hl
    N = len(db_names)
    AC, PR, RC, HL = AC/N, PR/N, RC/N, HL/N
    print('AC (%): {:.2f}\nPR (%): {:.2f}\nRC (%): {:.2f}\nHL    : {:.2f}'.format(
        100*AC, 100*PR, 100*RC, HL))

In [6]:
cbir('cnn_model')

AC (%): 59.52
PR (%): 76.29
RC (%): 73.71
HL    : 4.75
